In [1]:
# Workbook to practice mapping using plotly
# Data from ACLED (https://www.acleddata.com/)
# ACLED codebook found here: https://www.acleddata.com/wp-content/uploads/dlm_uploads/2017/10/ACLED_Codebook_2019FINAL_pbl.pdf

In [1]:
import os
import pandas as pd
import plotly.plotly as py
%matplotlib inline

In [2]:
full_df = pd.read_csv('../data/africa_data.csv')
full_df['country'].unique()

array(['Burkina Faso', 'Democratic Republic of Congo', 'Mali', 'Nigeria',
       'South Africa', 'Tunisia', 'Burundi', 'Ghana', 'Guinea',
       'Ivory Coast', 'Libya', 'Mozambique', 'Uganda', 'Cameroon',
       'Kenya', 'Niger', 'Somalia', 'Sudan', 'Egypt', 'Angola',
       'Central African Republic', 'Morocco', 'Madagascar', 'South Sudan',
       'Ethiopia', 'Rwanda', 'Chad', 'Malawi', 'Mauritania', 'Liberia',
       'Zambia', 'Algeria', 'Gabon', 'Senegal', 'Equatorial Guinea',
       'Benin', 'Tanzania', 'Zimbabwe', 'eSwatini', 'Sierra Leone',
       'Lesotho', 'Guinea-Bissau', 'Republic of Congo', 'Namibia',
       'Eritrea', 'Togo', 'Gambia', 'Botswana', 'Djibouti'], dtype=object)

In [3]:
full_df.columns.values

array(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty',
       'event_date', 'year', 'time_precision', 'event_type',
       'sub_event_type', 'actor1', 'assoc_actor_1', 'inter1', 'actor2',
       'assoc_actor_2', 'inter2', 'interaction', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'timestamp', 'iso3'], dtype=object)

In [4]:
full_df['source_scale'].unique()

array(['National', 'Other-National', 'Subnational', 'Other',
       'National-International', 'Regional', 'National-Regional',
       'Subnational-Regional', 'International', 'Other-Subnational',
       'Subnational-National', 'Other-International',
       'Regional-International', 'Local', 'Other-National-International',
       'Other-Regional', 'Other-Regional-National-International',
       'Subnational-National-International', 'Subnational-International',
       'National-Subnational', 'Natonal-International',
       'Other-Regional-International', 'National-international',
       'Other-Local', 'Local-Regional', 'Local-National',
       'Subnational-regional', 'National-Regional-International',
       'National-regional', 'Regional-international',
       'Local-International', 'Other-Local-International',
       'Local-Subnational', 'Local ', 'National ', 'other-International',
       'Other-national', 'Other-international',
       'Subnational-international', 'Subnational-Regiona

In [5]:
full_df['year'].unique()

array([2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009])

In [6]:
full_df['event_type'].unique()

array(['Explosions/Remote violence', 'Riots',
       'Violence against civilians', 'Battles'], dtype=object)

In [7]:
violent_events = ['Explosions/Remote violence', 'Battles', 'Violence against civilians']

In [8]:
congo = full_df.loc[(full_df['country'] == 'Democratic Republic of Congo') & (full_df['event_type'].isin(violent_events))]

In [9]:
congo_yearly_dfs = {}
for year in congo['year'].unique():
    congo_yearly_dfs[year] = congo.loc[congo['year'] == year]

In [10]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [11]:
yearly_traces = []
yearly_annotations = []

yearly_colors = {2010: "#56137A", 2011: "#39A2AE", 2012: "#55DBCB", 2013: "#75E4B3",
                 2014: "#ED7D3A", 2015: "#8CD867", 2016: "#33CFA5", 2017: "#2FBF71",
                 2018: "#28AFB0", 2019: "#F34D35", 2009: "#EE964B"}

for year in congo_yearly_dfs:
    yearly_traces.append((year, go.Scattermapbox(
                                            lat=congo_yearly_dfs[year]['latitude'],
                                            lon=congo_yearly_dfs[year]['longitude'],
                                            mode='markers',
                                            name = str(year),
                                            marker=go.scattermapbox.Marker(
                                                size=4,
                                                color=yearly_colors[year].lower()
                                        ),
                                            text=congo_yearly_dfs[year]['sub_event_type'],
                                        )
                        ))
    


mapbox_access_token = os.environ['MAPBOX_KEY']


layout = go.Layout(
    height = 600,
    margin = dict( t=0, b=0, l=0, r=0 ),
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=-4,
            lon=23
        ),
        pitch=0,
        zoom=4,
        style='dark'
    ),
)

updatemenus = list([dict(active=-1, buttons=list(), pad = {'r': 0, 't': 8},
                         x = 0.05,
                         xanchor = 'left',
                         y = 1.0,
                         yanchor = 'top',
                         bgcolor = '#AAAAAA',
                         bordercolor = '#FFFFFF',
                         font = dict(size=11, color='#000000'))])

for index, item in enumerate(yearly_traces):

    visibility_list = [False] * len(yearly_traces)
    visibility_list[index] = True
    updatemenus[0]['buttons'].append(dict(label = str(item[0]),
                                          method = 'update',
                                          args = [{'visible': visibility_list},
                                                   {'title': str(year)}]))

updatemenus[0]['buttons'].append(dict(label = 'All',
                                       method = 'update',
                                       args = [{'visible': [True] * len(yearly_traces)},
                                               {'title': 'All Years'}]))

updatemenus.append(dict(
                        buttons=list([
                            dict(
                                args=['mapbox.style', 'dark'],
                                label='Dark',
                                method='relayout'
                            ),                    
                            dict(
                                args=['mapbox.style', 'light'],
                                label='Light',
                                method='relayout'
                            ),
                            dict(
                                args=['mapbox.style', 'satellite'],
                                label='Satellite',
                                method='relayout'
                            ),
                            dict(
                                args=['mapbox.style', 'satellite-streets'],
                                label='Satellite with Streets',
                                method='relayout'
                            )                    
                        ]),
                        direction = 'up',
                        x = 0.75,
                        xanchor = 'left',
                        y = 0.05,
                        yanchor = 'bottom',
                        bgcolor = '#000000',
                        bordercolor = '#FFFFFF',
                        font = dict(size=11)
                    ))

data = [item[1] for item in yearly_traces]

annotations = list([
    dict(text='Violent Conflicts in DRC (scroll to zoom)', font=dict(color='#fe4a49',size=14), borderpad=10, 
         x=0.05, y=0.05, xref='paper', yref='paper', align='left', showarrow=False, bgcolor='black'),
    dict(text='Year', x=0.01, y=0.99, yref='paper', align='left', showarrow=False,font=dict(color='white',size=14))
])

layout['updatemenus'] = updatemenus
layout['annotations'] = annotations


figure = dict(data=data, layout=layout)
py.iplot(figure, filename='africa-conflict-dropdown')



/Users/shelbyruettiger/projects/conflict/venv/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [ ]:
yearly_traces = []
yearly_annotations = []

yearly_colors = {2010: "#56137A", 2011: "#39A2AE", 2012: "#55DBCB", 2013: "#75E4B3",
                 2014: "#ED7D3A", 2015: "#8CD867", 2016: "#33CFA5", 2017: "#2FBF71",
                 2018: "#28AFB0", 2019: "#F34D35", 2009: "#EE964B"}

for year in congo_yearly_dfs:
    yearly_traces.append((year, go.Scattermapbox(
                                            lat=congo_yearly_dfs[year]['latitude'],
                                            lon=congo_yearly_dfs[year]['longitude'],
                                            mode='markers',
                                            name = str(year),
                                            marker=go.scattermapbox.Marker(
                                                size=4,
                                                color=yearly_colors[year].lower()
                                        ),
                                            text=congo_yearly_dfs[year]['sub_event_type'],
                                        )
                        ))
    


mapbox_access_token = os.environ['MAPBOX_KEY']


layout = go.Layout(
    height = 600,
    margin = dict( t=0, b=0, l=0, r=0 ),
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=-4,
            lon=23
        ),
        pitch=0,
        zoom=4,
        style='dark'
    ),
)

updatemenus = list([dict(active=-1, buttons=list(), pad = {'r': 0, 't': 8},
                         x = 0.05,
                         xanchor = 'left',
                         y = 1.0,
                         yanchor = 'top',
                         bgcolor = '#AAAAAA',
                         bordercolor = '#FFFFFF',
                         font = dict(size=11, color='#000000'))])

for index, item in enumerate(yearly_traces):

    visibility_list = [False] * len(yearly_traces)
    visibility_list[index] = True
    updatemenus[0]['buttons'].append(dict(label = str(item[0]),
                                          method = 'update',
                                          args = [{'visible': visibility_list},
                                                   {'title': str(year)}]))

updatemenus[0]['buttons'].append(dict(label = 'All',
                                       method = 'update',
                                       args = [{'visible': [True] * len(yearly_traces)},
                                               {'title': 'All Years'}]))

updatemenus.append(dict(
                        buttons=list([
                            dict(
                                args=['mapbox.style', 'dark'],
                                label='Dark',
                                method='relayout'
                            ),                    
                            dict(
                                args=['mapbox.style', 'light'],
                                label='Light',
                                method='relayout'
                            ),
                            dict(
                                args=['mapbox.style', 'satellite'],
                                label='Satellite',
                                method='relayout'
                            ),
                            dict(
                                args=['mapbox.style', 'satellite-streets'],
                                label='Satellite with Streets',
                                method='relayout'
                            )                    
                        ]),
                        direction = 'up',
                        x = 0.75,
                        xanchor = 'left',
                        y = 0.05,
                        yanchor = 'bottom',
                        bgcolor = '#000000',
                        bordercolor = '#FFFFFF',
                        font = dict(size=11)
                    ))

data = [item[1] for item in yearly_traces]

annotations = list([
    dict(text='Violent Conflicts in DRC (scroll to zoom)', font=dict(color='#fe4a49',size=14), borderpad=10, 
         x=0.05, y=0.05, xref='paper', yref='paper', align='left', showarrow=False, bgcolor='black'),
    dict(text='Year', x=0.01, y=0.99, yref='paper', align='left', showarrow=False,font=dict(color='white',size=14))
])

layout['updatemenus'] = updatemenus
layout['annotations'] = annotations


figure = dict(data=data, layout=layout)
py.iplot(figure, filename='drc-conflict-year-dropdown')

